# A recommantion system made by Esra, Rafaella and Olivier

In [1]:
import pandas as pd
import numpy as np

In [2]:
tbasics_movies = pd.read_csv('/Users/rafaellaporto/BeCode/what-should-i-wacth/clean_data.csv')

In [3]:
tbasics_movies['imdb_id'] = tbasics_movies['tconst']

In [4]:
movie = pd.read_csv('/Users/rafaellaporto/BeCode/what-should-i-wacth/movies_metadata.csv', low_memory=False)

In [5]:
movie = movie[['imdb_id', 'title', 'overview', 'genres', 'popularity']]

In [6]:
movie.shape

(45466, 5)

In [7]:
df  = tbasics_movies.merge(movie, on='imdb_id')
df.shape

(39017, 39)

# Demographic Filtering
Weighted rating based Recommender

In [8]:
#Average Rating
C = df['averageRating'].mean()
print(C)

6.27130225286418


In [9]:
# Calculate the minimum number of votes required to be in the chart, m
m = df['numVotes'].quantile(0.75)
print(m)

7781.0


In [10]:
# Filter out all qualified movies into a new DataFrame
# for a movie to feature in the charts, it must have more votes than at least 90% of the movies in the list.
df2 = df.copy().loc[df['numVotes'] >= m]
df2.shape

(9756, 39)

In [11]:
df2.shape

(9756, 39)

In [12]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['numVotes']
    R = x['averageRating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [13]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
df2['score'] = df2.apply(weighted_rating, axis=1)

In [14]:
df2.head()

,tconst,primaryTitle,startYear,runtimeMinutes,averageRating,numVotes,genre_Action,genre_Adult,genre_Adventure,genre_Animation,...,genre_Talk-Show,genre_Thriller,genre_War,genre_Western,imdb_id,title,overview,genres,popularity,score
25,tt0004972,The Birth of a Nation,1915,195,6.2,24983,False,False,False,False,...,False,False,True,False,tt0004972,The Birth of a Nation,The Birth of A Nation is a silent film from 19...,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",5.113205,6.216933
47,tt0006864,Intolerance,1916,163,7.7,15734,False,False,False,False,...,False,False,False,False,tt0006864,Intolerance: Love's Struggle Throughout the Ages,"The story of a poor young woman, separated by ...","[{'id': 18, 'name': 'Drama'}]",4.282118,7.227251
77,tt0009968,Broken Blossoms,1919,90,7.3,10473,False,False,False,False,...,False,False,False,False,tt0009968,Broken Blossoms,Broken Blossoms is an American silent film fro...,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",3.185824,6.861504
88,tt0010323,The Cabinet of Dr. Caligari,1920,76,8.0,64645,False,False,False,False,...,False,True,False,False,tt0010323,The Cabinet of Dr. Caligari,The Cabinet of Dr. Caligari is eerie and expre...,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",7.191271,7.814279
97,tt0011237,The Golem,1920,76,7.2,7867,False,False,False,False,...,False,False,False,False,tt0011237,The Golem: How He Came Into the World,"In the 16th century, the Jews of Prague face p...","[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",1.52413,6.738203


In [15]:
#Sort movies based on score calculated above
df2 = df2.sort_values('score', ascending=False)

#Print the top 15 movies
df2[['title', 'numVotes', 'averageRating', 'score', 'startYear', 'runtimeMinutes', 'overview']]

,title,numVotes,averageRating,score,startYear,runtimeMinutes,overview
15141,The Shawshank Redemption,2664730,9.3,9.291182,1994,142,Framed in the 1940s for the double murder of h...
8009,The Godfather,1846637,9.2,9.187711,1972,175,"Spanning the years 1945 to 1955, a chronicle o..."
24019,The Dark Knight,2637712,9.0,8.991974,2008,152,Batman raises the stakes in his war on crime. ...
18015,The Lord of the Rings: The Return of the King,1837114,9.0,8.988491,2003,201,Aragorn is revealed as the heir to the ancient...
14667,Schindler's List,1349325,9.0,8.984355,1993,195,The true story of how businessman Oskar Schind...
...,...,...,...,...,...,...,...
25022,The Hottie & The Nottie,38426,1.9,2.636103,2008,91,"Nate moves to L.A. to track down Cristabel, th..."
19956,Superbabies: Baby Geniuses 2,31139,1.5,2.453893,2004,88,"Following on from the plot of the last movie, ..."
6362,Manos: The Hands of Fate,36521,1.6,2.420446,1966,70,A family gets lost on the road and stumbles up...
27529,Disaster Movie,91869,1.9,2.241326,2008,87,"In DISASTER MOVIE, the filmmaking team behind ..."


# Content-Based Recommender
Plot Description Based Recommender

In [16]:
df = df.reset_index(drop=True)

In [26]:
df.head()

,tconst,primaryTitle,startYear,runtimeMinutes,averageRating,numVotes,genre_Action,genre_Adult,genre_Adventure,genre_Animation,...,genre_Sport,genre_Talk-Show,genre_Thriller,genre_War,genre_Western,imdb_id,title,overview,genres,popularity
0,tt0000574,The Story of the Kelly Gang,1906,70,6.0,797,True,False,True,False,...,False,False,False,False,False,tt0000574,The Story of the Kelly Gang,Just as Fritz Lang’s Metropolis (1927) is test...,"[{'id': 36, 'name': 'History'}, {'id': 28, 'na...",0.290549
1,tt0002101,Cleopatra,1912,100,5.1,536,False,False,False,False,...,False,False,False,False,False,tt0002101,Cleopatra,The fabled queen of Egypt's affair with Roman ...,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",0.142542
2,tt0002130,Dante's Inferno,1911,71,7.0,2988,False,False,True,False,...,False,False,False,False,False,tt0002130,L'inferno,Loosely adapted from Dante's Divine Comedy and...,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",0.801412
3,tt0002461,The Life and Death of King Richard III,1912,55,5.7,287,False,False,False,False,...,False,False,False,False,False,tt0002461,Richard III,Shakespeare's tragedy of the hump-backed Duke ...,"[{'id': 18, 'name': 'Drama'}]",0.05131
4,tt0002844,Fantômas: In the Shadow of the Guillotine,1913,54,6.9,2328,False,False,False,False,...,False,False,False,False,False,tt0002844,Fantômas: In the Shadow of the Guillotine,"For the arch-criminal Fantômas, no crime is to...","[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",0.791727


In [17]:
df.shape

(39017, 39)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english') #Remove all english stop words such as 'the', 'a'
df['overview'] = df['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(df['overview'])

size = tfidf_matrix.shape
#We see that 70731 different words were used to describe the 39017 movies in our dataset.

In [19]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [20]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [21]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]


In [22]:
get_recommendations('The Godfather')

8601       The Godfather: Part II
13421     The Godfather: Part III
30958                  Blood Ties
14522            Household Saints
11135           Start Liquidation
23275                    Election
924      A Mother Should Be Loved
17865           Short Sharp Shock
3772            The Sign of Venus
7338                 Violent City
Name: title, dtype: object

In [23]:
get_recommendations('The Lord of the Rings')

18015        The Lord of the Rings: The Return of the King
18546                                 Magic Christmas Tree
34268                                                 Wolf
34956                                       A Fighting Man
21955                                        Underclassman
16837    The Lord of the Rings: The Fellowship of the Ring
18016                The Lord of the Rings: The Two Towers
3072                                               Trapped
1306                                  Daughter of Shanghai
38436                                                Bling
Name: title, dtype: object

In [24]:
get_recommendations('Pineapple Express')

14438                  Cop and ½
3085                  The Window
29843            Daydream Nation
2844     Last of the Wild Horses
17651         Mark Strikes Again
12057                   No Mercy
20270                   Showtime
978             She Loves Me Not
35759                   Survivor
8411                     Sssssss
Name: title, dtype: object

In [25]:
get_recommendations('This Is the End')

7601          Born to Win
35740          It Follows
26110    The Green Hornet
33545        Almost Human
15790        Commandments
16546    Niagara, Niagara
15565       Parallel Sons
16831      Kissing a Fool
19663            Intimacy
32391        Abel's Field
Name: title, dtype: object

# Content-Based Recommender #2
Credits, Genres, and Keywords Based Recommender

In [ ]:
def recommend(movie):
    index = movies[movies['title'] == movie].index[0]
    similar_movies = sorted(enumerate(similarity[index]), key=lambda x: x[1], reverse=True)[1:6]

    recommended_movies = []
    recommended_movies_poster = []
    for i in similar_movies:
        movie_id = movies.iloc[i[0]].movie_id

        recommended_movies.append(movies.iloc[i[0]].title)
        recommended_movies_poster.append(fetch_poster(movie_id))
    return recommended_movies , recommended_movies_poster

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

In [27]:
import pickle
pickle.dump(df , open('movies.pkl' , 'wb'))

In [ ]:
pickle.dump(cosine_sim, open('similarity.pkl' , 'wb'))